In [1]:
import random
import numpy as np
import Parsing_midi as pm
import Init_chromo as init
import count_CE as ce
import mido
from mido import Message, MidiFile, MidiTrack
from collections import Counter
import copy

from itertools import combinations # product: 排列

/home/hsuan/env1/lib/python3.6/site-packages/pydub/utils.py:165: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


In [2]:
"""
## import function

- Parsing_midi
    - get_midi
    - get_pitch
    - parse_midi
    - separate_phrase

- Init_chromo
    - shift_note
    - merge_duplicate_note
    - extend_duration
"""
print()

### Seperate phrase
- one bar = 1920 (4 beats)
- sliding window = 7680 (4 bar)
- Parameters:
    - base
    - midi list
    - threshold

In [3]:
midi_list = pm.get_midi('MidiSample/iswear2.mid',0)

## seperate input into 4 bar
def separate_song(song, threshold):
    a = song
    time = 0; t = []; t2 = []; n = []; n2 = []
    for i in range(len(a)):
        time += a[i].time 
        if time <= threshold:
#             time += a[i].time       
            t.append(a[i])
            n.append(a[i].note%12)
            
        else:
            t2.append(t)  # sepatate bar
            n2.append(n)  # index of note
            ## reset
            time = 0 
            t = []
            n = []
            time += a[i].time
            t.append(a[i])
            n.append(a[i].note%12)
    return n2, t2

In [4]:
## transform to C major
for i in range(len(midi_list)):
    midi_list[i].note = midi_list[i].note -2 # shift

In [5]:
## get midi message
pitch_ary = [[0,1,0], [0,-1,2], [0,1,4], [-1,0,11], [1,0,1], [-1,0,3],[1,0,5]]

pitch_ary = np.array(pitch_ary)

def pitch_trans(seq):
    pitch_list = ['C','D','E','F','G','A','B']
    return pitch_list[seq]

def get_key(CE, pitch_ary):
    key = []
    for i in range(len(pitch_ary)):
        key.append(abs(pitch_ary[i][0]-int(CE[0])) + abs(pitch_ary[i][1]-int(CE[1])) + \
                  abs(pitch_ary[i][2]-int(CE[2])))
    predict_key = pitch_trans(key.index(min(key)))
    return predict_key

note_msg, midi_msg = separate_song(midi_list, 1920)

### Evolutionary algorithmn
- selection
- crossover
- mutation
- fitness

In [6]:
## MMAIN -- Initialization
pop_list = [0,1,2,3,4] #目前是5個initial choromosome
candidate = list(combinations(pop_list, 2))

## get midi clip for testing
pops = midi_msg
parent = pops[2]  # change here for testing
parent

[<message note_on channel=0 note=83 velocity=0 time=239>,
 <message note_on channel=0 note=84 velocity=120 time=241>,
 <message note_on channel=0 note=84 velocity=0 time=479>,
 <message note_on channel=0 note=84 velocity=120 time=1>,
 <message note_on channel=0 note=84 velocity=0 time=239>,
 <message note_on channel=0 note=84 velocity=98 time=1>,
 <message note_on channel=0 note=84 velocity=0 time=239>,
 <message note_on channel=0 note=84 velocity=96 time=1>,
 <message note_on channel=0 note=84 velocity=0 time=239>,
 <message note_on channel=0 note=76 velocity=100 time=1>,
 <message note_on channel=0 note=76 velocity=0 time=239>,
 <message note_on channel=0 note=77 velocity=105 time=1>]

In [7]:
len(candidate)

10

In [8]:
## Initialize (at least 50)
pop1 = init.shift_note(parent, 3)             # 移調
pop2 = init.shift_note(parent, 5)             # 移調
pop4 = init.merge_duplicate_note(parent)      # 重複的切分音直接轉為連音
pop5 = init.change_duration(parent, 120, 480, 3) # 延長音 16分音符->一拍
pop6 = init.change_duration(parent, 240, 120, 3) # 縮短音 8分音符->16分音符
pop_list = [pop1, pop2, pop4, pop5, pop6]
pops=[]
for i in range(len(pop_list)):
    pops.append(pop_list[i])

def trans_note_list(song):
    note_list=[]
    for i in range(len(song)):
        note_list.append(song[i].note)
    return note_list
    
## Selection
def select(parent=pops):  # select one phrase
    print("selection")
    pop_list = [0,1,2,3,4]
#     idx = np.random.choice(pop_list, replace=True, p=[0.2,0.2,0.2,0.2,0.2])  
    return parent[idx]


## Crossover
"""
Parameter:
crossover_rate
crossover_point
"""
def crossover(parent, pop):
    print("crossover...")
    crossover_point = random.randint(0,len(parent))
    pop_midi1 = parent[ :crossover_point] + pop[crossover_point: ]
    pop_midi2 = pop[ :crossover_point] + parent[crossover_point: ]
    return pop_midi1, pop_midi2

    
## Mutation
"""
Parameter:
mute_rate
mute_point
mutate_list
"""
def mutation(child):
    print("mutation...")
    mute_point = random.randint(0,len(child)-1)  #選擇一點做突變
    mutate_note = np.random.choice(7, 1, replace=False, p=[0.2, 0.2, 0.25, 0.05, 0.25, 0.025, 0.025]) #突變之機率
    mutate_list = [child[mute_point].note + int(mutate_note), 0]  #選擇突變或是直接刪除  ##TODO: 相加後可能超過12->跨一八度
    child[mute_point].note = int(np.random.choice(mutate_list, 1, p=[0.7, 0.3]))
    return child
   
    
## Evaluation
"""
fitness function
pitch
chord
"""

## base on CE get the chord name 
chord_list = [[0,4,7], [2,5,9], [4,7,11], [5,9,0], [7,11,2], [9,0,4], [11,2,5]]
pitch_list = ['C','D','E','F','G','A','B']

def fitness(midi_msg, note_msg): # type: list
    print("fitness evaluate...")
    score = 0
    scale_list = [0,2,4,5,7,9,11]
    ## count CE
    
    ## Get first and last note
    first_note = midi_msg[0].note
    last_note = midi_msg[-1].note
    
    ## Pitch
    for i in range(len(midi_msg)-1):
        if abs(midi_msg[i].note - midi_msg[i+1].note) == 4: ## major third
            score += 1 
        if abs(midi_msg[i].note - midi_msg[i+1].note) == 7: ## perfect fifth
            score += 1
        if abs(midi_msg[i].note - midi_msg[i+1].note) > 7:  ## big jump note
            score = score - 6
            
        # get chord (by CE) -> get note -> match
        key = get_key(note_msg, pitch_ary)
        chord = chord_list[pitch_list.index(key)]
        if midi_msg[i].note == chord[0]:## chord root note
            score += 4
        if midi_msg[i].note == chord[1]:## 2nd note
            score += 4
        if midi_msg[i].note == chord[2]:## 3rd note
            score += 4
        if midi_msg[i].note == first_note and midi_msg[i].note == chord[0]:
            score += 3
        if midi_msg[i].note == first_note and midi_msg[i].note == chord[1]:
            score += 2
        if midi_msg[i].note == first_note and midi_msg[i].note == chord[2]:
            score += 2
        if midi_msg[i].note == first_note and midi_msg[i].note not in chord: # not in chord note
            score -= 1   
        if midi_msg[i].note == last_note and midi_msg[i].note == chord[0]:
            score += 3
        if midi_msg[i].note == last_note and midi_msg[i].note == chord[1]:
            score += 2
        if midi_msg[i].note == last_note and midi_msg[i].note == chord[2]:
            score += 2
        if midi_msg[i].note == last_note and midi_msg[i].note not in chord: # not in chord note
            score -= 1
        if midi_msg[i].note not in chord: # note is not a chord note
            score -= 3
        if midi_msg[i].note % 12 not in scale_list:  # note not in the scale
            score -= 8
            
    ## Chord
    return score, midi_msg

In [14]:
len(candidate)

10

In [15]:
## MAIN FUNCTION

MUTATION_RATE = 0.5
CROSSOVER_RATE = 0.8
GENERATIONS = 100
FITNESS_VALUE = 0.5
count = 0; score_list = []; midi_list = []

while count < GENERATIONS:
    for i in range(len(candidate)):
        
        # 任選一組candicate
        parent = pops[candidate[i][0]]
        pop = pops[candidate[i][1]]

        # fitness 1
        parent_result_note = trans_note_list(parent)
        pop_result_note = trans_note_list(pop)
        parent_score, parent_midi_output = fitness(parent, parent_result_note)
        pop_score, pop_midi_output = fitness(pop, pop_result_note)

        # crossover 
        crossover_rate = round(random.uniform(0, 1),1)
        if crossover_rate < CROSSOVER_RATE:
            child1, child2 = crossover(parent, pop)
        else:
            child1 = parent 
            child2 = pop

        # mutation
        mute_rate = round(random.uniform(0, 1),1)    
        if mute_rate < MUTATION_RATE:    
            result_child1 = mutation(child1)
            result_child2 = mutation(child2)
        else:
            result_child1 = child1
            result_child2 = child2

        # fitness 2
        result_note1 = trans_note_list(result_child1)
        result_note2 = trans_note_list(result_child2)
        output_score1, midi_output1 = fitness(result_child1, result_note1)
        output_score2, midi_output2 = fitness(result_child2, result_note2)

        # compare output_score1 vs parent_result_note and output_score2 vs pop_result_note
        if output_score1 > parent_score:
            parent = midi_output1
            score_list.append(output_score1)
            midi_list.append(parent)
        else:
            score_list.append(parent_score)
            midi_list.append(parent)
            
        if output_score2 > pop_score:
            pop = midi_output2
            score_list.append(output_score2)
            midi_list.append(pop)    
        else:
            score_list.append(pop_score)
            midi_list.append(pop)
            
    count += 1
    print(count)
print("finish")   

fitness evaluate...
fitness evaluate...
crossover...
fitness evaluate...
fitness evaluate...
fitness evaluate...
fitness evaluate...
mutation...
mutation...
fitness evaluate...
fitness evaluate...
fitness evaluate...
fitness evaluate...
crossover...
fitness evaluate...
fitness evaluate...
fitness evaluate...
fitness evaluate...
crossover...
fitness evaluate...
fitness evaluate...
fitness evaluate...
fitness evaluate...
crossover...
fitness evaluate...
fitness evaluate...
fitness evaluate...
fitness evaluate...
mutation...
mutation...
fitness evaluate...
fitness evaluate...
fitness evaluate...
fitness evaluate...
crossover...
mutation...
mutation...
fitness evaluate...
fitness evaluate...
fitness evaluate...
fitness evaluate...
crossover...
fitness evaluate...
fitness evaluate...
fitness evaluate...
fitness evaluate...
crossover...
fitness evaluate...
fitness evaluate...
fitness evaluate...
fitness evaluate...
crossover...
fitness evaluate...
fitness evaluate...
1
fitness evaluate...
fi

mutation...
fitness evaluate...
fitness evaluate...
15
fitness evaluate...
fitness evaluate...
crossover...
mutation...
mutation...
fitness evaluate...
fitness evaluate...
fitness evaluate...
fitness evaluate...
crossover...
fitness evaluate...
fitness evaluate...
fitness evaluate...
fitness evaluate...
crossover...
fitness evaluate...
fitness evaluate...
fitness evaluate...
fitness evaluate...
crossover...
fitness evaluate...
fitness evaluate...
fitness evaluate...
fitness evaluate...
crossover...
mutation...
mutation...
fitness evaluate...
fitness evaluate...
fitness evaluate...
fitness evaluate...
fitness evaluate...
fitness evaluate...
fitness evaluate...
fitness evaluate...
crossover...
mutation...
mutation...
fitness evaluate...
fitness evaluate...
fitness evaluate...
fitness evaluate...
crossover...
fitness evaluate...
fitness evaluate...
fitness evaluate...
fitness evaluate...
fitness evaluate...
fitness evaluate...
fitness evaluate...
fitness evaluate...
crossover...
fitness e

mutation...
fitness evaluate...
fitness evaluate...
fitness evaluate...
fitness evaluate...
mutation...
mutation...
fitness evaluate...
fitness evaluate...
29
fitness evaluate...
fitness evaluate...
crossover...
mutation...
mutation...
fitness evaluate...
fitness evaluate...
fitness evaluate...
fitness evaluate...
mutation...
mutation...
fitness evaluate...
fitness evaluate...
fitness evaluate...
fitness evaluate...
crossover...
mutation...
mutation...
fitness evaluate...
fitness evaluate...
fitness evaluate...
fitness evaluate...
crossover...
mutation...
mutation...
fitness evaluate...
fitness evaluate...
fitness evaluate...
fitness evaluate...
crossover...
fitness evaluate...
fitness evaluate...
fitness evaluate...
fitness evaluate...
crossover...
mutation...
mutation...
fitness evaluate...
fitness evaluate...
fitness evaluate...
fitness evaluate...
fitness evaluate...
fitness evaluate...
fitness evaluate...
fitness evaluate...
crossover...
fitness evaluate...
fitness evaluate...
fit

fitness evaluate...
fitness evaluate...
fitness evaluate...
fitness evaluate...
crossover...
mutation...
mutation...
fitness evaluate...
fitness evaluate...
fitness evaluate...
fitness evaluate...
mutation...
mutation...
fitness evaluate...
fitness evaluate...
fitness evaluate...
fitness evaluate...
fitness evaluate...
fitness evaluate...
fitness evaluate...
fitness evaluate...
crossover...
fitness evaluate...
fitness evaluate...
fitness evaluate...
fitness evaluate...
crossover...
mutation...
mutation...
fitness evaluate...
fitness evaluate...
fitness evaluate...
fitness evaluate...
fitness evaluate...
fitness evaluate...
fitness evaluate...
fitness evaluate...
crossover...
mutation...
mutation...
fitness evaluate...
fitness evaluate...
fitness evaluate...
fitness evaluate...
crossover...
fitness evaluate...
fitness evaluate...
42
fitness evaluate...
fitness evaluate...
crossover...
fitness evaluate...
fitness evaluate...
fitness evaluate...
fitness evaluate...
crossover...
mutation..

fitness evaluate...
fitness evaluate...
crossover...
mutation...
mutation...
fitness evaluate...
fitness evaluate...
fitness evaluate...
fitness evaluate...
crossover...
mutation...
mutation...
fitness evaluate...
fitness evaluate...
fitness evaluate...
fitness evaluate...
crossover...
mutation...
mutation...
fitness evaluate...
fitness evaluate...
56
fitness evaluate...
fitness evaluate...
crossover...
mutation...
mutation...
fitness evaluate...
fitness evaluate...
fitness evaluate...
fitness evaluate...
crossover...
mutation...
mutation...
fitness evaluate...
fitness evaluate...
fitness evaluate...
fitness evaluate...
crossover...
mutation...
mutation...
fitness evaluate...
fitness evaluate...
fitness evaluate...
fitness evaluate...
crossover...
fitness evaluate...
fitness evaluate...
fitness evaluate...
fitness evaluate...
crossover...
mutation...
mutation...
fitness evaluate...
fitness evaluate...
fitness evaluate...
fitness evaluate...
mutation...
mutation...
fitness evaluate...
f

fitness evaluate...
fitness evaluate...
fitness evaluate...
crossover...
fitness evaluate...
fitness evaluate...
67
fitness evaluate...
fitness evaluate...
crossover...
mutation...
mutation...
fitness evaluate...
fitness evaluate...
fitness evaluate...
fitness evaluate...
mutation...
mutation...
fitness evaluate...
fitness evaluate...
fitness evaluate...
fitness evaluate...
crossover...
mutation...
mutation...
fitness evaluate...
fitness evaluate...
fitness evaluate...
fitness evaluate...
crossover...
fitness evaluate...
fitness evaluate...
fitness evaluate...
fitness evaluate...
crossover...
mutation...
mutation...
fitness evaluate...
fitness evaluate...
fitness evaluate...
fitness evaluate...
crossover...
mutation...
mutation...
fitness evaluate...
fitness evaluate...
fitness evaluate...
fitness evaluate...
crossover...
fitness evaluate...
fitness evaluate...
fitness evaluate...
fitness evaluate...
crossover...
fitness evaluate...
fitness evaluate...
fitness evaluate...
fitness evalu

fitness evaluate...
crossover...
fitness evaluate...
fitness evaluate...
fitness evaluate...
fitness evaluate...
crossover...
mutation...
mutation...
fitness evaluate...
fitness evaluate...
fitness evaluate...
fitness evaluate...
crossover...
mutation...
mutation...
fitness evaluate...
fitness evaluate...
fitness evaluate...
fitness evaluate...
crossover...
fitness evaluate...
fitness evaluate...
fitness evaluate...
fitness evaluate...
fitness evaluate...
fitness evaluate...
fitness evaluate...
fitness evaluate...
crossover...
mutation...
mutation...
fitness evaluate...
fitness evaluate...
fitness evaluate...
fitness evaluate...
crossover...
fitness evaluate...
fitness evaluate...
fitness evaluate...
fitness evaluate...
crossover...
fitness evaluate...
fitness evaluate...
fitness evaluate...
fitness evaluate...
fitness evaluate...
fitness evaluate...
fitness evaluate...
fitness evaluate...
fitness evaluate...
fitness evaluate...
82
fitness evaluate...
fitness evaluate...
mutation...
mu

fitness evaluate...
fitness evaluate...
fitness evaluate...
fitness evaluate...
crossover...
fitness evaluate...
fitness evaluate...
fitness evaluate...
fitness evaluate...
crossover...
fitness evaluate...
fitness evaluate...
fitness evaluate...
fitness evaluate...
crossover...
mutation...
mutation...
fitness evaluate...
fitness evaluate...
fitness evaluate...
fitness evaluate...
mutation...
mutation...
fitness evaluate...
fitness evaluate...
fitness evaluate...
fitness evaluate...
crossover...
fitness evaluate...
fitness evaluate...
94
fitness evaluate...
fitness evaluate...
crossover...
fitness evaluate...
fitness evaluate...
fitness evaluate...
fitness evaluate...
crossover...
mutation...
mutation...
fitness evaluate...
fitness evaluate...
fitness evaluate...
fitness evaluate...
crossover...
fitness evaluate...
fitness evaluate...
fitness evaluate...
fitness evaluate...
crossover...
mutation...
mutation...
fitness evaluate...
fitness evaluate...
fitness evaluate...
fitness evaluate.

In [20]:
print(len(score_list))
print(len(midi_list))

2000
2000


In [21]:
midi_list[0]

[<message note_on channel=0 note=0 velocity=0 time=239>,
 <message note_on channel=0 note=25 velocity=120 time=241>,
 <message note_on channel=0 note=0 velocity=0 time=479>,
 <message note_on channel=0 note=6 velocity=120 time=1>,
 <message note_on channel=0 note=5 velocity=0 time=239>,
 <message note_on channel=0 note=7 velocity=98 time=1>,
 <message note_on channel=0 note=9 velocity=0 time=239>,
 <message note_on channel=0 note=0 velocity=96 time=1>,
 <message note_on channel=0 note=2 velocity=0 time=239>,
 <message note_on channel=0 note=20 velocity=100 time=1>,
 <message note_on channel=0 note=0 velocity=0 time=239>,
 <message note_on channel=0 note=0 velocity=105 time=1>]

### Output to midi file

In [363]:
def to_midi_file(inputfile, program, outputfile):
    ## initialize
    mid = MidiFile()
    track = MidiTrack()
    mid.tracks.append(track)
    ## add message
    track.append(Message('program_change', program=program, time=0))
    for i in range(len(inputfile)):
        track.append(Message('note_on', note=inputfile[i].note, velocity=inputfile[i].velocity, time=inputfile[i].time))
    mid.save(outputfile)
    return True

In [367]:
to_midi_file(midi_list[0], 18, 't.mid')

True

---